<a href="https://colab.research.google.com/github/mbaliu-treino/Desenvolve/blob/main/LEARN_C_NLP_Word2Vec_Treinamento_de_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=orange><b>Word2Vec: treinamento de Word Embedding</b></font>

<font color=gray size=2><a href="https://colab.research.google.com/drive/1X6z7zpGcas2mGn-JSFbV7nN5nBeeMW_q">Arquivo Colab</a></font>

<ul><font size=2 color=gray><b>FICHA TÉCNICA</b>
<li><a href="https://cursos.alura.com.br/course/word2vec-treinamento-word-embedding"><font size=2 color=gray>Word2Vec: treinamento de Word Embedding</a>
<li>Carga Horária: 10 h
<li>Instrutora: Thiago G Santos
<li>Data de Início: 09-2022
</ul>

<hr color=gray><br>

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)


<h3><b>Conteúdo / Aprendizagem:</b></h3>

- 

# <font color=orange>RESUMO</font>

<a href=https://docs.google.com/spreadsheets/d/1tTygYlq9r7nkUsw9a25N5_z57-de_59tSBCIVPw6KUw><font size=2 color=gray>ROTEIROS Data Science</font></a>

* <font color=orange><b>

# <font color=orange>CURSO</font>

**Projeto**: Neste curso, vamos criar nossa própria representação Word2Vec, para criar um classificador de artigos de jornais, usando apenas os títulos de cada reportagem. Treinado o modelo Word2Vec e o classificador, ambos modelos serão disponibilizados em uma aplicação Web para simular um modelos de machine learning em produção.

* [Paper de Tomas Mikolov](https://arxiv.org/pdf/1301.3781.pdf)
* [Embedding Project do TensorFloe](https://projector.tensorflow.org/)

<a href="https://github.com/alura-cursos/word2vec_treinamento/tree/aula06"><font size=2; color=gray>Material de referencial do curso</a></font>

## <font color=orange>Carga de Dados</font>

A base de dados será a base de conhecimento para o modelo, fornecendo um conhecimento previamente já consolidado que servirá para o treinamento. No NLP, essa base de dados é conhecimento como *corpus textual*.



In [ ]:
# Para o output do Colab quebrar as linhas longas
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import pandas as pd

In [ ]:
uri_treino = 'https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/treino.csv'
uri_teste = 'https://caelum-online-public.s3.amazonaws.com/1638-word-embedding/teste.csv'

In [ ]:
dados_treino = pd.read_csv(uri_treino)
dados_teste = pd.read_csv(uri_teste)

In [ ]:
dados_treino.sample(5)

,title,text,date,category,subcategory,link
16575,Ruy Castro apresenta programa de rádio com rar...,"A partir deste domingo (2), o escritor e colun...",2017-01-07,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/07...
34990,"Observado pelo presidente, Ganso se destaca em...",Criticado pela falta de apresentações convince...,2015-03-04,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/04/1...
42051,Criminosos explodem caixa eletrônico em posto ...,Um grupo de criminosos explodiu na madrugada d...,2015-04-02,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/02...
15064,China inicia processo na OMC contra Estados Un...,Um dia após expirar o prazo para que os demais...,2016-12-12,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/12/1...
89634,Cleiton Xavier e Egídio fazem tratamento em di...,Os jogadores do Palmeiras ganharam folga nesta...,2015-02-11,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/11/1...


In [ ]:
dados_treino.shape

(90000, 6)

In [ ]:
dados_teste.shape

(20513, 6)

## Tratamento dos dados


### SpaCy

Aqui será visto a biblioteca SpaCy para o NLP. Ela é uma biblioteca *open source* desenvolvida para a aplicação de NLP na industria.

> Esta biblioteca foi projetada especificamente para uso em produção e ajuda a criar aplicações que processam e “compreendem” grandes volumes de textos. Alguns exemplos de utilização são para extração de informações, pré-processamento e aprendizado profundo.

**VANTAGENS**: Ela facilita o processamento de dados por ao realizar a tokenização das frases, ela já realiza o *tageamento* delas, ou seja, a classificação das palavras (stopword, alpha)

In [ ]:
# O Colab já possui o spacy, mas não o modelo em português
!python -m spacy download pt_core_news_sm

2022-09-12 19:34:14.808679: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 3.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import spacy

In [ ]:
nlp = spacy.load('pt_core_news_sm')

### Estruturas do Spacy

In [ ]:
texto = 'Rio de Janeiro é uma cidade maravilhosa.'

doc = nlp(texto)

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc[0]

Rio

In [ ]:
for i in doc:
    print(i)

Rio
de
Janeiro
é
uma
cidade
maravilhosa
.


In [ ]:
# Entidade nomeada (ner)
doc.ents

(Rio de Janeiro,)

In [ ]:
# Tags
doc[0].is_stop

False

In [ ]:
doc[1].is_stop

True

## Pre-Processamento

Para o pré=processamento dos textos para o treinamento do modelo Word2Vec podemos usar como referência o projeto do NILC-USP de [Words Embeddings](https://github.com/nathanshartmann/portuguese_word_embeddings/blob/master/preprocessing.py).

> "NILC-Embeddings é um repositório destinado ao armazenamento e compartilhamento de vetores de palavras (do inglês, word embeddings) gerados para a Língua Portuguesa. O objetivo é fomentar e tornar acessível recursos vetoriais prontos para serem utilizados nas tarefas de Processamento da Linguagem Natural e Aprendizado de Máquina. O repositório traz vetores gerados a partir de um grande córpus do português do Brasil e português europeu, de fontes e gêneros variados. Foram utilizados dezessete córpus diferentes, totalizando 1,395,926,282 tokens." [[fonte]](http://www.nilc.icmc.usp.br/embeddings)


**TIPOS DE TRATAMENTOS RECOMENDADOS**

* Caso dos caracteres (upper case, lower case)

In [ ]:
# Função de tratamento proposto pelo curso
def trata_textos(doc):

    tokens_validos = []
    for token in doc:
        token_is_valid = not (token.is_stop) and (token.is_alpha)
        if token_is_valid:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 3:
        return " ".join(tokens_validos)
    else:
        return None

In [ ]:
# Função de tratamento proposto pelo curso
def trata_textos_(doc):

    tokens_validos = []
    for token in doc:
        token_is_valid = not (token.is_stop) and (token.is_alpha)
        if token_is_valid:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 3:
        return " ".join(tokens_validos)
    else:
        return None

In [ ]:
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [ ]:
# %%time
# textos_para_tratamento = dados_treino['title']
# textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=-1)]

In [ ]:
# %%time
# # Sem processamento paralelo
# textos_para_tratamento = dados_treino['title']
# textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=1)]

In [ ]:
%%time
# Com tratamento do case em generator
textos_para_tratamento = (texto.lower() for texto in dados_treino['title'])
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=1)]

CPU times: user 2min 5s, sys: 2.17 s, total: 2min 7s
Wall time: 2min 16s


In [ ]:
titulos_tratados = pd.DataFrame({'titulo': textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


## Treinamento do Word2Vec

Para o treinamento do modelo iremos usar a biblioteca `Gensim` que já traz uma estrutura de arquitetura de modelo Word2Vec construída, permitindo focar-se na parametrização do modelo (ajuste dos hiperparâmetros).

> Os hiperparâmetros são parâmetros que configuram a forma que seu modelo será treinado, por isso são passados antes da fase de treinamento.

### Hiperparâmetros

* **CBOW ou SkipGram**: `sg` 1 para skip-gram e 0 para CBOW.
* **Contexto**: para a utilização do contexto é preciso indicar a extensão dele. O hiperparâmetro `window` é a quantidade de palavras antes e depois a serem consideradas.
* **Vetor final**: o tamanho do vetor final é especificado pelo parâmetro `size`.
* **Palavras raras**: quando uma palavra poucas vezes no *Corpus*, são chamadas de palavras raras. Elas podem ser tanto erros de digitação, quanto palavras raramente usadas. Para especificar o número limite para a classificação é usado o hiperparâmetro `min_count`.
* **Taxa de aprendizagem da rede neural**: a taxa de aprendizagem é o quanto a função de custo será ajustada a cada época. `alpha`.
* **Decaimento da taxa de aprendizado**: é o valor de adaptação da taxa de aprendizagem. `min_alpha`.

In [ ]:
# Construção do modelo Word2Vec - hiperparametrização
from gensim.models import Word2Vec

w2v_modelo_cbow = Word2Vec(
    sg=0,
    window=2,
    size=300,
    min_count=5,
    alpha=0.03,
    min_alpha=0.007
    )

### Vocabulário

In [ ]:
%%time
# Construção do dicionário para o modelo
print(len(titulos_tratados))
titulos_tratados_limpos = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados_limpos))

lista_lista_tokens = [titulo.split(' ') for titulo in titulos_tratados_limpos['titulo']]

w2v_modelo_cbow.build_vocab(lista_lista_tokens)

90000
81890
CPU times: user 3.7 s, sys: 79.4 ms, total: 3.78 s
Wall time: 3.76 s


In [ ]:
# Verificação do status
import logging

logging.basicConfig(format='%(asctime)s : - %(message)s', level=logging.INFO)

w2v_modelo_cbow = Word2Vec(
    sg=0,
    window=2,
    size=300,
    min_count=5,
    alpha=0.03,
    min_alpha=0.007
    )

# progress_per: a cada quantas iterações será fornecido um logging
w2v_modelo_cbow.build_vocab(lista_lista_tokens, progress_per=5000)

### Treinamento

In [ ]:
# lista de todos os métodos do modelo
dir(w2v_modelo_cbow)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [ ]:
w2v_modelo_cbow.corpus_count

81890

In [ ]:
%%time
w2v_modelo_cbow.train(
    sentences=lista_lista_tokens, 
    total_examples=w2v_modelo_cbow.corpus_count,
    epochs=30)

CPU times: user 1min 18s, sys: 417 ms, total: 1min 19s
Wall time: 46.8 s


(14375579, 15975420)

In [ ]:
# Acompanhamento do LOSS

from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
    def __init__(self):
       self.epoch = 0

    def on_epoch_end(self, model):
       loss = model.get_latest_training_loss()
       if self.epoch == 0:
           print('Loss após a época {}: {}'.format(self.epoch, loss))
       else:
           print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
       self.epoch += 1
       self.loss_previous_step = loss

w2v_modelo_cbow.train(lista_lista_tokens,
                total_examples=w2v_modelo_cbow.corpus_count,
                epochs = 30,
                compute_loss = True,
                callbacks=[callback()])


Loss após a época 0: 158919.65625
Loss após a época 1: 145967.40625
Loss após a época 2: 131811.65625
Loss após a época 3: 143050.28125
Loss após a época 4: 119695.25
Loss após a época 5: 128598.5625
Loss após a época 6: 117480.0625
Loss após a época 7: 118883.625
Loss após a época 8: 112398.125
Loss após a época 9: 103673.625
Loss após a época 10: 111996.625
Loss após a época 11: 97924.625
Loss após a época 12: 89518.625
Loss após a época 13: 97927.25
Loss após a época 14: 85448.0
Loss após a época 15: 78867.0
Loss após a época 16: 82162.125
Loss após a época 17: 89632.5
Loss após a época 18: 83465.75
Loss após a época 19: 86028.0
Loss após a época 20: 80119.25
Loss após a época 21: 75751.75
Loss após a época 22: 65985.0
Loss após a época 23: 76307.25
Loss após a época 24: 59289.75
Loss após a época 25: 66221.0
Loss após a época 26: 73099.0
Loss após a época 27: 68077.5
Loss após a época 28: 62990.25
Loss após a época 29: 62369.75


(14375241, 15975420)

### Avaliação Qualitativa

In [ ]:
w2v_modelo_cbow.wv.most_similar('google')

[('apple', 0.5686640739440918),
 ('amazon', 0.5243529081344604),
 ('facebook', 0.5122028589248657),
 ('uber', 0.5095587372779846),
 ('airbnb', 0.5026870965957642),
 ('volkswagen', 0.4959263801574707),
 ('fbi', 0.4950755536556244),
 ('yahoo', 0.48570019006729126),
 ('software', 0.48143675923347473),
 ('waze', 0.4802655875682831)]

In [ ]:
w2v_modelo_cbow.wv.most_similar('barcelona')

[('barça', 0.58600914478302),
 ('chelsea', 0.5792363286018372),
 ('bayern', 0.5661696195602417),
 ('psg', 0.5624327659606934),
 ('madrid', 0.5565645694732666),
 ('munique', 0.5323404669761658),
 ('liverpool', 0.5309878587722778),
 ('lazio', 0.5296096801757812),
 ('juventus', 0.5268446803092957),
 ('messi', 0.5260409116744995)]

In [ ]:
w2v_modelo_cbow.wv.most_similar('microsoft')

[('unilever', 0.6302778720855713),
 ('amazon', 0.5911853313446045),
 ('viajante', 0.5587636828422546),
 ('sky', 0.5560619831085205),
 ('tesla', 0.5396549105644226),
 ('walmart', 0.5346637964248657),
 ('canais', 0.5332616567611694),
 ('varejista', 0.5294968485832214),
 ('linkedin', 0.5278860926628113),
 ('sony', 0.5112634301185608)]

In [ ]:
w2v_modelo_cbow.wv.most_similar('bmw')

[('toyota', 0.613348662853241),
 ('renault', 0.5565720796585083),
 ('honda', 0.5401179790496826),
 ('embraer', 0.5335556268692017),
 ('sky', 0.5214617252349854),
 ('bélgica', 0.5075830221176147),
 ('tesla', 0.5050802230834961),
 ('chrysler', 0.5025763511657715),
 ('mercedes', 0.5013681054115295),
 ('gm', 0.5002136826515198)]

### Modelo - Skip Gram

In [ ]:
%%time
# Hiperparâmetros
w2v_modelo_sg = Word2Vec(
    sg=1,
    window=5,  # dizem que é melhor para contextos maiores
    size=300,
    min_count=5,
    alpha=0.03,
    min_alpha=0.007
    )

# Vocabulário
w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

# Treinamento
w2v_modelo_sg.train(
    sentences=lista_lista_tokens, 
    total_examples=w2v_modelo_sg.corpus_count,
    epochs=30,
    compute_loss = True,
    callbacks=[callback()])

Loss após a época 0: 2076794.0
Loss após a época 1: 1546981.75
Loss após a época 2: 1169528.75
Loss após a época 3: 1118603.5
Loss após a época 4: 1128271.5
Loss após a época 5: 1030755.0
Loss após a época 6: 964937.5
Loss após a época 7: 806726.0
Loss após a época 8: 878747.0
Loss após a época 9: 717549.0
Loss após a época 10: 833038.0
Loss após a época 11: 811635.0
Loss após a época 12: 705693.0
Loss após a época 13: 689201.0
Loss após a época 14: 711162.0
Loss após a época 15: 695146.0
Loss após a época 16: 641641.0
Loss após a época 17: 592900.0
Loss após a época 18: 492404.0
Loss após a época 19: 535302.0
Loss após a época 20: 467776.0
Loss após a época 21: 463268.0
Loss após a época 22: 458428.0
Loss após a época 23: 470320.0
Loss após a época 24: 459272.0
Loss após a época 25: 400754.0
Loss após a época 26: 398066.0
Loss após a época 27: 433624.0
Loss após a época 28: 404510.0
Loss após a época 29: 400258.0
CPU times: user 3min 11s, sys: 591 ms, total: 3min 12s
Wall time: 1min 5

(14375040, 15975420)

In [ ]:
w2v_modelo_sg.wv.most_similar('google')

[('reguladores', 0.4379476308822632),
 ('buffett', 0.418307900428772),
 ('android', 0.41497597098350525),
 ('waze', 0.39129090309143066),
 ('facebook', 0.38948163390159607),
 ('yahoo', 0.3703403174877167),
 ('concorda', 0.368610680103302),
 ('warren', 0.3662242293357849),
 ('apple', 0.3653578758239746),
 ('anunciantes', 0.3526855707168579)]

### Salvando modelo

In [ ]:
w2v_modelo_cbow.wv.save_word2vec_format('modelo_cbow.txt', binary=False)
w2v_modelo_sg.wv.save_word2vec_format('modelo_skipgram.txt', binary=False)

### Resumo CBOW x SG

| CARACTERÍSTICA | CBOW | SKIP-GRAM |
|----------------|------|-----------|
|Tempo de treinamento| Menor | Maior|
|Tamanho do contexto| Menor | Maior |
| Loss           | Maior | Menor |

# Classificação

## 

In [ ]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])  # otimização do nlp

In [ ]:
# # AMBIENTE
# !python -m spacy download pt_core_news_sm

# # MODELOS WORD2VEC (em formato txt)
# w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_cbow.txt")
# w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_skipgram.txt")

# # DADOS
# # artigo_treino = pd.read_csv("/content/drive/My Drive/curso_word2vec/treino.csv")
# # artigo_teste = pd.read_csv("/content/drive/My Drive/curso_word2vec/teste.csv")
# dados_treino = pd.read_csv(uri_treino)
# dados_teste = pd.read_csv(uri_teste)

In [ ]:
# Tokenizador do texto
def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            texto_lower = token.text.lower()
            tokens_validos.append(texto_lower)
    return  tokens_validos

# Combinação Vetorial do título
def combinacao_de_vetores_por_soma(palavras, modelo):
    vetor_resultante = np.zeros((1,300))
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            pass  # ignore qualquer erro de chave
    return vetor_resultante

# União dos vetores
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)
    return matriz

In [ ]:
# # TESTES DAS FUNÇÕES
# texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
# tokens = tokenizador(texto)
# print(tokens)

# vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
# print(vetor_texto.shape)
# print(vetor_texto)

In [ ]:
# Modelos em formato Word2Vec Obj 
# Sem otimizar o processamento de spacy (7min + )
# Otimizando ( ? + 1,5min)
matriz_vetores_treino_cbow = matriz_vetores(dados_treino['title'], w2v_modelo_cbow.wv)
matriz_vetores_teste_cbow = matriz_vetores(dados_teste['title'], w2v_modelo_cbow.wv)

print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape) 

(90000, 300)
(20513, 300)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(x_treino, y_treino, x_teste, y_teste):
    RL = LogisticRegression(max_iter=800)
    RL.fit(x_treino, y_treino)

    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)

    return RL

In [ ]:
%%time
RL_cbow = classificador( 
                        matriz_vetores_treino_cbow,
                        dados_treino['category'],
                        matriz_vetores_teste_cbow,
                        dados_teste['category'])

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.75      6103
   cotidiano       0.63      0.81      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.82      0.22       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.82      0.79      0.80     20513



In [ ]:
%%time
matriz_vetores_treino_sg = matriz_vetores(dados_treino['title'], w2v_modelo_sg.wv)
matriz_vetores_teste_sg = matriz_vetores(dados_teste['title'], w2v_modelo_sg.wv)

RL_sg = classificador( 
    matriz_vetores_treino_sg,
    dados_treino['category'],
    matriz_vetores_teste_sg,
    dados_teste['category'])

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.14      0.86      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.74      0.85      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



## Salvando classificador

In [ ]:
import pickle

with open('rl_cbow.pkl', 'wb') as file_pkl:
    pickle.dump(RL_cbow, file_pkl)

with open('rl_sg.pkl', 'wb') as file_pkl:
    pickle.dump(RL_sg, file_pkl)

# Disponibilizando o Modelo

Iremos usar uma estrutura web feita previamente com o Flask. 

**1. REPOSITÓRIO**: Para baixar a estrutura fornecida é recomendado usar o `git clone` para ter uma cópia do repositório no local.

* [Repositório](https://github.com/tgcsantos/classificador_de_artigos)

**2. BIBLIOTECAS**: Depois é preciso construir o ambiente para a aplicação, baixando todas as bibliotecas necessárias.

```cmd
python -m env env
env\scripts\activate
pip install -r requirements.txt
```

**3. EXECUÇÃO DO SERVICE**: podemos executar o serviço app e abrir a página com `http://localhost:5000`.

```cmd
python app.py
```

## Estrutura do Projeto

PASTAS
* `data`: contém os dados usados no treinamento e teste do modelo Word2Vec
* `models`: contém os modelos de Word2Vec, CBOW e SkipGram, e os modelos de classificação logística.
* `notebooks`: os notebooks usados no curso.
* `statics` e `templates`: para construir a página HTML
* `app.py`: é o aquivo que estrutura o projeto Flask
* `utils.py`: possui funções, já criadas no curso, para auxiliar na transformação vetorial das frases inputadas.

```python
# Author: Thiago Gonçalves Claro dos Santos <thiago.gsantos03@gmail.com>
# Github e Twitter: @tgcsantos

import pickle
from gensim.models import KeyedVectors
from flask import Flask, request, render_template

from utils import tokenizador, combinacao_de_vetores_por_soma

app = Flask(__name__, template_folder="templates")

@app.before_first_request
def load_models():
    
    global classificador
    global w2v_modelo

    w2v_dir = "models/modelo_skipgram.txt"
    classificador_dir = "models/rl_sg.pkl"
    w2v_modelo = KeyedVectors.load_word2vec_format(w2v_dir)
 
    with open(classificador_dir, "rb") as f:
        classificador = pickle.load(f)


@app.route('/')
def home():
    return render_template('index.html')


@app.route('/predict',methods=['POST'])
def predict():
    
    # Obtenção do dado inputado pelo usuário
    titulo = next(request.form.values())
    
    # Tokenização da frase fornecida
    titulo_tokens = tokenizador(titulo)
    # Vetorização combinada da frase
    titulo_vetor = combinacao_de_vetores_por_soma(titulo_tokens, w2v_modelo)
    # Classificação do vetor
    titulo_categoria = classificador.predict(titulo_vetor)

    # Apresentação do resultado
    output = titulo_categoria[0].capitalize()

    return render_template('index.html',
                            title='Título: {}'.format(titulo), 
                            category='Categoria: {}'.format(output))

if __name__ == "__main__":
    app.run(debug=True)

```

> A função predict() classifica o título e retorna render_template, que vai renderizar o arquivo HTML index.html e alterar os valores de title e category.